In [1]:
import numpy as np
from scipy.stats import entropy
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, TransfoXLTokenizer, TFTransfoXLLMHeadModel
from scipy.special import softmax
import torch
import random
import string

In [2]:
def get_distribution(model_info, model_name, context, joint_vocab):

        model, tokenizer = model_info[model_name]

        input = tokenizer(context,return_tensors='tf')
        outputs = model(input)

        ids = range(0,len(outputs))
        vocab = tokenizer.convert_ids_to_tokens(ids)

        final_vocab = vocab & joint_vocab

        # get id's corresponding to the final_vocab
        
        # only get the IDS of things that are in joint_vocab
        id_list = tokenizer.convert_tokens_to_ids(final_vocab)
        print(id_list)

        # at this point, take only outputs over what's in the joint vocab 
        
        probabilities = softmax(outputs[0])
        

        distr_dict = dict(zip(vocab, probabilities[0][0]))

        return distr_dict


In [5]:
model_info = {"GPT2": (TFGPT2LMHeadModel.from_pretrained("gpt2"),GPT2Tokenizer.from_pretrained("gpt2")), 
              "TransformerXL": (TFTransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103'),TransfoXLTokenizer.from_pretrained('transfo-xl-wt103'))}

curr_context = "What"
gpt2_dict = get_distribution(model_info, "GPT2", curr_context, {})
txl_dict = get_distribution(model_info, "TransformerXL", curr_context, {})

joint_vocab = gpt2_dict.keys() & txl_dict.keys()

print(joint_vocab)

p = get_distribution(model_info, "GPT2", context, joint_vocab)
q = get_distribution(model_info, "TransformerXL", context, joint_vocab)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All model checkpoint weights were used when initializing TFTransfoXLLMHeadModel.

All the weights of TFTransfoXLLMHeadModel were initialized from the model checkpoint at transfo-xl-wt103.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFTransfoXLLMHeadModel for predictions without further training.


{'599', 'exclusive', 'Pa', 'Base', 'Having', '554', 'biz', 'Br', 'oming', 'starter', '490', 'Reloaded', '338', '427', 'dust', 'Recommended', 'Offset', 'option', 'negative', 'antics', 'nos', 'assment', 'boa', 'Speaking', 'CA', 'ebook', 'Looking', 'hunter', 'private', '1027', 'Capt', 'pit', 'tailed', 'SH', '700', 'op', 'orb', 'powered', 'Hz', 'this', 'fur', 'Mart', 'Appearances', 'rounder', 'elected', '649', 'Suddenly', 'brother', 'Al', 'Rose', 'cigarettes', 'DN', 'helm', 'SL', 'Mania', 'twitch', 'those', 'icy', 'British', 'Destroy', '949', 'Foot', '452', 'sis', 'paying', 'Anim', 'ILS', 'EAR', 'tall', 'esh', '040', 'fter', 'Shock', 'e', 'zen', '81', 'pictured', 'pure', 'death', 'automatic', 'groups', 'initial', 'GG', 'Ter', 'Intel', 'Â£', 'judicial', '790', 'warm', 'Fuck', '222', 'EW', 'way', 'evidence', 'Done', 'Builder', 'Breaking', '245', 'Gs', 'haired', 'Obama', '031', 'layer', 'correct', 'sex', 'AND', '996', 'christ', 'Mode', 'Day', 'Els', 'Summer', 'World', 'Congratulations', 'powe

NameError: name 'context' is not defined

In [4]:
def js(p, q):

    intersection = p.keys() & q.keys()
    p = {k:v for k,v in p.items() if k in intersection}
    q = {k:v for k,v in q.items() if k in intersection}

    p = [v for k, v in sorted(p.items())]
    q = [v for k, v in sorted(q.items())]

    p = np.asarray(p)
    q = np.asarray(q)
    # normalize
    p /= p.sum()
    q /= q.sum()
    m = (p + q) / 2
    return (entropy(p, m) + entropy(q, m)) / 2

In [5]:
def auto_regressive(model_info, curr_context, num_return_seqs, current_len, max_len, total_js, joint_vocab):

    if current_len == max_len:
        return total_js / current_len

    distrs = {}
    highest = {}
    for model_name in ['GPT2','TransformerXL']:
        model, tokenizer = model_info[model_name]
        next_word_distr = get_distribution(model_info, model_name, curr_context, joint_vocab)
        distrs[model_name] = next_word_distr

    A = distrs['GPT2']
    B = distrs['TransformerXL']
    # average the two distributions
    avg_distr = {x: (A.get(x, 0) + B.get(x, 0))/2 for x in set(A).intersection(B)}
    
    js_dict = {}
    for i in range(0,5):
        n = random.randint(0,len(avg_distr))
        new_word = avg_distr.items()[n][0]
        print("NEW WORD", new_word)
        print("CURR PRE-NEW CONTEXT", curr_context)
        new_context =  curr_context + new_word
        print("NEW CONTEXT", new_context)
        p = get_distribution(model_info, 'GPT2', new_context, joint_vocab)
        q = get_distribution(model_info,'TransformerXL', new_context, joint_vocab)
        print(p,q)
        js_result = js(p,q)
        js_dict[new_word] = js_result


    highest_js_word = sorted(js_dict.items(), key=lambda x: x[1], reverse=True)[0][0]
    print("highest JS word", highest_js_word)
    curr_context = curr_context + highest_js_word

    p = get_distribution(model_info,'GPT2', curr_context, joint_vocab)
    q = get_distribution(model_info,'TransformerXL', curr_context, joint_vocab)

    total_js += js(p,q)
    print("CURR CONTEXT", curr_context, "JS", total_js)
    # then autoregressive again on this new current context
    return auto_regressive(model_info, curr_context, num_return_seqs, current_len + 1, max_len , total_js)

                 

In [8]:
model_info = {"GPT2": (TFGPT2LMHeadModel.from_pretrained("gpt2"),GPT2Tokenizer.from_pretrained("gpt2")), 
              "TransformerXL": (TFTransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103'),TransfoXLTokenizer.from_pretrained('transfo-xl-wt103'))}

curr_context = "What"
gpt2_dict = get_distribution(model_info, "GPT2", curr_context, {})
txl_dict = get_distribution(model_info, "TransformerXL", curr_context, {})

joint_vocab = gpt2_dict.keys() & txl_dict.keys()

auto_regressive(model_info, curr_context, 50, 1, 15, 0, joint_vocab)

All model checkpoint weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All model checkpoint weights were used when initializing TFTransfoXLLMHeadModel.

All the weights of TFTransfoXLLMHeadModel were initialized from the model checkpoint at transfo-xl-wt103.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFTransfoXLLMHeadModel for predictions without further training.


NameError: name 'softmax' is not defined